In [157]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from itertools import chain
from collections import OrderedDict
from itertools import combinations 

# Recursive

In [158]:
'''
corpus =['I1,I2,I5',
'I2,I4',
'I2,I3',
'I1,I2,I4',
'I1,I3',
'I2,I3',
'I1,I3',
'I1,I2,I3,I5',
'I1,I2,I3']

'''
corpus=['I1,I2,I5',
'I2,I3,I4',
'I3,I4',
'I1,I2,I3,I4']

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
index_col=['T'+str(i+1)  for i in range(len(corpus))]
print(index_col)
print(X.toarray())
print(index_col)
print(X.toarray())

['i1', 'i2', 'i3', 'i4', 'i5']
['T1', 'T2', 'T3', 'T4']
[[1 1 0 0 1]
 [0 1 1 1 0]
 [0 0 1 1 0]
 [1 1 1 1 0]]
['T1', 'T2', 'T3', 'T4']
[[1 1 0 0 1]
 [0 1 1 1 0]
 [0 0 1 1 0]
 [1 1 1 1 0]]


In [159]:
columns=vectorizer.get_feature_names()
columns

['i1', 'i2', 'i3', 'i4', 'i5']

In [160]:
#L[k].apply(lambda row: row[row == 1].index, axis=1).to_list()

#L[k].columns[L[k].eq(True).any()]

In [161]:
columns
C={}
L={}
candidates=None
min_sup=2
def getMinSupport(my_table,min_sup,candidates_temp):
  if candidates_temp is None:    
    df_L=my_table.loc[:, (my_table .sum(axis=0) >= min_sup)]      #Sum of columns is greater than minimum support
    df_L=df_L[df_L.sum(axis=1) >= min_sup]                        #Sum of rows is greater than minimum support
    return df_L
  else:
    rows_singleton=my_table.index.to_list() #Get min support rows
    cols_singleton=L[1].columns.to_list()   # Get min support singleton columns
    #print('rows_singleton ' ,rows_singleton)
    #print('cols_singleton ' ,cols_singleton)

    new_df=L[1][cols_singleton] 
    #print('new_df')
    #print(new_df)
    for itemset in candidates_temp:
      combination_string = ",".join(itemset)
      itemset_as_list=[]
      for item in itemset:
        itemset_as_list.append(item)
      print(itemset_as_list)
      new_df[combination_string] = new_df[itemset_as_list].all(axis=1)

    #print(new_df)
    new_df.drop(cols_singleton,axis=1,inplace=True)

    df_L=new_df.loc[:, (new_df .sum(axis=0) >= min_sup)]  #Sum of columns is greater than minimum support
    df_L=new_df[df_L.sum(axis=1) >= min_sup]              #Sum of rows is greater than minimum support
    
  return df_L

def getCandidateSets(arr,tuple_size,iteration):   
  temp_itemtuple_list=[]
  temp=list(combinations(arr,2))  
  for item in temp:
    mylist=(item[0]+',' +item[1]).split(',')
    t=list(dict.fromkeys(mylist))
    if len(t) == iteration+1:
      valid_tuple=t
      temp_itemtuple_list.append(valid_tuple)
  return temp_itemtuple_list 

In [162]:
k=1
C[k]=pd.DataFrame(data=X.toarray(), index=index_col ,columns=columns)
C[k]

,i1,i2,i3,i4,i5
T1,1,1,0,0,1
T2,0,1,1,1,0
T3,0,0,1,1,0
T4,1,1,1,1,0


In [163]:
L[k]=getMinSupport(C[k],min_sup,candidates)
L[k]

,i1,i2,i3,i4
T1,1,1,0,0
T2,0,1,1,1
T3,0,0,1,1
T4,1,1,1,1


In [164]:
candidates=getCandidateSets(L[k].columns.to_list(),2,k)
candidates
k=k+1
L[k]=getMinSupport(L[k-1],min_sup,candidates)
L[k]

['i1', 'i2']
['i1', 'i3']
['i1', 'i4']
['i2', 'i3']
['i2', 'i4']
['i3', 'i4']


,"i1,i2","i1,i3","i1,i4","i2,i3","i2,i4","i3,i4"
T2,False,False,False,True,True,True
T4,True,True,True,True,True,True


In [165]:
candidates=getCandidateSets(L[k].columns.to_list(),2,k)
candidates
k=k+1
L[k]=getMinSupport(L[k-1],min_sup,candidates)
L[k]

['i1', 'i2', 'i3']
['i1', 'i2', 'i4']
['i1', 'i2', 'i3']
['i1', 'i2', 'i4']
['i1', 'i3', 'i4']
['i1', 'i3', 'i2']
['i1', 'i3', 'i4']
['i1', 'i4', 'i2']
['i1', 'i4', 'i3']
['i2', 'i3', 'i4']
['i2', 'i3', 'i4']
['i2', 'i4', 'i3']


,"i1,i2,i3","i1,i2,i4","i1,i3,i4","i1,i3,i2","i1,i4,i2","i1,i4,i3","i2,i3,i4","i2,i4,i3"
T2,False,False,False,False,False,False,True,True
T4,True,True,True,True,True,True,True,True


In [166]:
candidates=getCandidateSets(L[k].columns.to_list(),2,k)
candidates
k=k+1
L[k]=getMinSupport(L[k-1],min_sup,candidates)
L[k]

['i1', 'i2', 'i3', 'i4']
['i1', 'i2', 'i3', 'i4']
['i1', 'i2', 'i3', 'i4']
['i1', 'i2', 'i3', 'i4']
['i1', 'i2', 'i3', 'i4']
['i1', 'i2', 'i3', 'i4']
['i1', 'i2', 'i4', 'i3']
['i1', 'i2', 'i4', 'i3']
['i1', 'i2', 'i4', 'i3']
['i1', 'i2', 'i4', 'i3']
['i1', 'i2', 'i4', 'i3']
['i1', 'i3', 'i4', 'i2']
['i1', 'i3', 'i4', 'i2']
['i1', 'i3', 'i4', 'i2']
['i1', 'i3', 'i4', 'i2']
['i1', 'i3', 'i2', 'i4']
['i1', 'i3', 'i2', 'i4']
['i1', 'i3', 'i2', 'i4']
['i1', 'i3', 'i2', 'i4']
['i1', 'i4', 'i2', 'i3']
['i1', 'i4', 'i2', 'i3']
['i1', 'i4', 'i2', 'i3']
['i1', 'i4', 'i3', 'i2']
['i1', 'i4', 'i3', 'i2']


,"i1,i2,i3,i4","i1,i2,i4,i3","i1,i3,i4,i2","i1,i3,i2,i4","i1,i4,i2,i3","i1,i4,i3,i2"


In [167]:
if (len(L[k].index) ==0):
  print('Terminated  generaion of itemsets')

Terminated  generaion of itemsets
